Load Data

In [ ]:
import pandas as pd
personal_data = pd.read_csv('https://storage.googleapis.com/datalynn-datasets/Interview_Challenge/JPMorgan/JPMorgan_credit_resks_prediction/personal_data.csv')
credit_data = pd.read_csv('https://storage.googleapis.com/datalynn-datasets/Interview_Challenge/JPMorgan/JPMorgan_credit_resks_prediction/credit_data.csv')
default_data = pd.read_csv('https://storage.googleapis.com/datalynn-datasets/Interview_Challenge/JPMorgan/JPMorgan_credit_resks_prediction/default_data.csv')


## 3. Questions for the Data Challenge##



###1. Data Manipulation (SQL)###







**Question**: Suppose we want to evaluate the overall creditworthiness of our clients. However, for some customers, the Credit_Score information in the Credit_Data table is missing. Can you generate a query that provides a list of all CustomerIDs where the Credit_Score data is not yet available?

In [ ]:
import sqlite3

# Create an in-memory SQLite database
conn = sqlite3.connect(':memory:')

# Store the DataFrames in the database as tables
personal_data.to_sql('personal_data', conn, index=False)
credit_data.to_sql('credit_data', conn, index=False)
default_records.to_sql('default_records', conn, index=False)

# Checking for Null values
query = '''
SELECT
    CustomerID
FROM
    Credit_Data
WHERE
    Credit_Score IS NULL;

'''

conn.execute(query)
conn.commit()

# Retrieve the updated dataframe from the database
credit_data = pd.read_sql_query("SELECT * FROM Credit_Data", conn)

# Close the database connection
conn.close()

**Question**: Using the Credit_Data table, can you calculate the cumulative loan amount for each customer, ordered by the loan term? What do you observe about the patterns of borrowing over time for customers?

**Answer**:

In [ ]:
import sqlite3

# Create an in-memory SQLite database
conn = sqlite3.connect(':memory:')

# Store the DataFrames in the database as tables
personal_data.to_sql('personal_data', conn, index=False)
credit_data.to_sql('credit_data', conn, index=False)
default_records.to_sql('default_records', conn, index=False)

# Checking for Null values
query = '''
SELECT COUNT(*)
FROM Credit_Data
WHERE CustomerID IS NULL OR Loan_Term IS NULL OR Loan_Amount IS NULL;
'''

# Checking for duplicates
query = '''
SELECT CustomerID, Loan_Term, COUNT(*)
FROM Credit_Data
GROUP BY CustomerID, Loan_Term
HAVING COUNT(*) > 1;
'''

# Calculate the cumulative loan amount for each customer ordered by the loan term
query = '''
SELECT
    CustomerID,
    Loan_Term,
    Loan_Amount,
    SUM(Loan_Amount) OVER (
        PARTITION BY CustomerID
        ORDER BY Loan_Term
        ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
    ) AS Cumulative_Loan_Amount
FROM Credit_Data
ORDER BY CustomerID, Loan_Term;
'''

conn.execute(query)
conn.commit()

# Retrieve the updated dataframe from the database
credit_data = pd.read_sql_query("SELECT * FROM Credit_Data", conn)

# Close the database connection
conn.close()

###2. Feature Processing and Feature Engineering (Python)###



**Question**: In the context of predicting credit default risks for potential loan borrowers, we have numerical features like 'Income', 'Loan_Amount', 'Credit_Score', and 'Age', and categorical features like 'Occupation', 'Education', 'Loan_Type'. How would you prepare these features for model training, specifically handling the categorical variables?

In [ ]:
# Merge the three dataframes based on 'CustomerID'
df = personal_data.merge(credit_data, on='CustomerID', how='inner')
df = df.merge(default_records, on='CustomerID', how='inner')

# Data Understanding
for col in ['Occupation', 'Education', 'Loan_Type','Loan_Term']:
    print(f"{col}:\n{df[col].value_counts()}\n{'-'*50}")

# Handle Missing Values
df['Occupation'] = df['Occupation'].fillna('Unknown')

# Encoding
df = pd.get_dummies(df, columns=['Occupation', 'Education', 'Loan_Type','Loan_Term'])

Occupation:
Lawyer      261
Engineer    254
Teacher     252
Doctor      233
Name: Occupation, dtype: int64
--------------------------------------------------
Education:
PhD              279
Graduate         249
High School      244
Undergraduate    228
Name: Education, dtype: int64
--------------------------------------------------
Loan_Type:
Home        346
Personal    330
Auto        324
Name: Loan_Type, dtype: int64
--------------------------------------------------
Loan_Term:
Medium    374
Short     345
Long      281
Name: Loan_Term, dtype: int64
--------------------------------------------------


###3. Modeling Metrics (Python)###





**Question**: In credit default prediction, different types of errors have different costs. For instance, predicting a customer will not default when they actually will (False Negative) might be more costly than predicting a customer will default when they actually will not (False Positive). Given these considerations, what kind of performance metric would you suggest that we use instead of traditional ones like accuracy? Explain your choice and how it might be beneficial in this particular business context. Additionally, write a Python function to calculate this metric given the true labels and the model predictions.

###4. Machine Learning Modeling and Model Optimization(Python)###




**Question**: For our credit default prediction task, assume you have decided to initially start with Logistic Regression. With this model selection, what pre-training manipulation would implement? Why would you implement feature scaling for this model? Why is scaling important? Except Logistic Regression, what other models would require scaling? Please provide Python code to demonstrate.

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

# Assume df is the DataFrame containing the data

# Let's split the data into features and target variable
X = df.drop('Default', axis=1)
y = df['Default']

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the StandardScaler
scaler = StandardScaler()

# Fit and transform the training data
X_train_scaled = scaler.fit_transform(X_train)

# Transform the test data
X_test_scaled = scaler.transform(X_test)

# Initialize and fit the Logistic Regression model
log_reg = LogisticRegression()
log_reg.fit(X_train_scaled, y_train)

# Generate predictions on the training and testing data
y_train_pred = log_reg.predict(X_train_scaled)
y_test_pred = log_reg.predict(X_test_scaled)

# Calculate the F1 scores
f1_train = f1_score(y_train, y_train_pred)
f1_test = f1_score(y_test, y_test_pred)

print("Training F1 Score:", f1_train)
print("Testing F1 Score:", f1_test)

Training F1 Score: 0.0
Testing F1 Score: 0.0


**Question**: Assume you have decided to optimize your model by using XGBoost instead of Logistic Regression. XGBoost is known for its speed and performance but it has quite a few hyperparameters to tune. How do you approach this? And what's your strategy to ensure you're not overfitting the model?

In [ ]:
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier

# Initialize XGBoost classifier
model = XGBClassifier()

# Define the hyperparameters
param_grid = {
    'n_estimators': [100, 200, 500],
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 5, 7],
    'colsample_bytree': [0.3, 0.5, 0.7],
    'gamma': [0, 0.1, 0.2]
}

# Set up the grid search
grid_search = GridSearchCV(model, param_grid, cv=5, scoring='roc_auc')

# Conduct the grid search
grid_search.fit(X_train, y_train)

# Get the best parameters
best_params = grid_search.best_params_

# Fit the model with the best parameters to the full training set
model = XGBClassifier(**best_params)
model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.7, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=0.2, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.05, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=5, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=500, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [ ]:
# Generate predictions on the training and testing data
y_train_pred = model.predict(X_train_scaled)
y_test_pred = model.predict(X_test_scaled)

# Calculate the F1 scores
f1_train = f1_score(y_train, y_train_pred)
f1_test = f1_score(y_test, y_test_pred)

print("Training F1 Score:", f1_train)
print("Testing F1 Score:", f1_test)

Training F1 Score: 0.017699115044247787
Testing F1 Score: 0.044444444444444446


**Question**: Assume with XGBoost, our model's AUC-ROC score is high, but when we deploy it in production, the actual positive predictive value (precision) is much lower than expected. What could explain this discrepancy and how would you investigate?

###5. Business Insights (Python)###



**Question**:Given the model results, which features appear to be the most significant predictors of loan default? How would you communicate these insights to business stakeholders in order to shape future lending strategies?

